In [1]:
#dependencies and setup
import pandas as pd
import numpy as np

#import api key
from api_keys import weather_key

#incorportating citipy
from citipy import citipy

#output file
output_data_file = "data/storm_data.csv"

# Range of latitudes and longitudes
lat_range = (64, 20)
lng_range = (-154, -65)

#pull in csv
data_df = pd.read_csv("data/noaa_storm_data.csv")

In [2]:
data_df.head()

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EVENT_NARRATIVE,EPISODE_NARRATIVE,ABSOLUTE_ROWNUMBER
0,866322,CARTERET CO.,TRUTTNEYS LNDG,11/1/2019,15,Tornado,,EF0,0,0,...,1,NNW,TRUTTNEYS LNDG,34.7394,-77.1094,34.7411,-77.1053,A brief tornado occurred near the intersection...,A strong cold front moved across Eastern North...,1
1,859389,MARTIN CO.,WAVELAND,11/5/2019,1620,Tornado,,EF0,0,0,...,2,ENE,WAVELAND,27.2140,-80.1729,27.2123,-80.1766,Martin County Beach Patrol reported a waterspo...,A strong thunderstorm produced a waterspout ju...,2
2,864077,MOBILE CO.,PRICHARD,11/7/2019,1335,Tornado,,EF0,0,0,...,0,SSE,PRICHARD,30.7248,-88.0548,30.7260,-88.0486,The storm survey team surveyed the areas aroun...,Thunderstorms moved across the area during the...,3
3,857862,FRANKLIN PAR.,BASKIN,11/26/2019,1803,Tornado,,EF2,1,1,...,4,NE,BASKIN,32.2351,-91.7783,32.3184,-91.7095,This tornado began on Roger Miller Road as it ...,"Warm, moist air was in place across northeast ...",4
4,857944,CLAIBORNE PAR.,MAHAN,11/26/2019,1820,Tornado,,EF1,0,0,...,2,NNE,MAHAN,32.9017,-93.0423,32.9076,-93.0343,An EF-1 tornado with estimated maximum winds a...,A deep southwesterly flow aloft ahead of a lar...,5


In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

528

In [4]:
url = "api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}"
# List of city data
city_data_df = []
# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")
# Create counters
record_count = 1
set_count = 1
# Loop through all the cities in our list
for x, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes
    if (x % 50 == 0 and x >= 50):
        set_count += 1
        record_count = 0
    # Create endpoint URL with each city
    city_url = url + "&q=" + city
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))
    # Add 1 to the record count
    record_count += 1
    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        # Parse out the begin_location
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_wind = city_weather["wind"]["speed"]
        city_date = city_weather["dt"]
        country = country["US"]
        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Wind Speed": city_wind,
                          "Date": city_date,
                         "Country": country})
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | constitucion
City not found. Skipping...
Processing Record 2 of Set 1 | fortuna
City not found. Skipping...
Processing Record 3 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 4 of Set 1 | clifton
City not found. Skipping...
Processing Record 5 of Set 1 | dolbeau
City not found. Skipping...
Processing Record 6 of Set 1 | san quintin
City not found. Skipping...
Processing Record 7 of Set 1 | peachland
City not found. Skipping...
Processing Record 8 of Set 1 | woodward
City not found. Skipping...
Processing Record 9 of Set 1 | sitka
City not found. Skipping...
Processing Record 10 of Set 1 | pacific grove
City not found. Skipping...
Processing Record 11 of Set 1 | nautla
City not found. Skipping...
Processing Record 12 of Set 1 | macomb
City not found. Skipping...
Processing Record 13 of Set 1 | homer
City not found. Skipping...
Processing Record 14 of Set 1 | kodiak
City not 

In [13]:
tornado_df = data_df.loc[(data_df["Country"]=="US")]
tornado_df

KeyError: 'Country'

In [72]:
us_df = pd.DataFrame(city_data_df)
us_df

""


In [59]:

#  Get the indices of cities that have humidity over 100%.

dirty_city_df = us_df.loc[us_df["country"]==["US"]]
dirty_city_df

AttributeError: 'list' object has no attribute 'loc'